# Time series

A time series is a series with dates as index.

   1. [Creation & use](#creation)
   2. [Data manipulation](#manipulation)
   3. [Examples](#examples)

Note: see what is a date/time in Python if you haven't: [Introduction to DateTime](../lesson2 Deeper in Python/21 datetime.ipynb)

Ref: http://pandas.pydata.org/pandas-docs/stable/timeseries.html

On peut choisir d'indexer son tableau par un index chronologique. Dans ce cas certaines opérations
liées au temps deviennent possibles.

Note: regardez ce qu'est une date/heure dans Python si vous ne l'avez pas encore fait : [Introduction à DateTime](../lesson2 Deeper in Python/21 datetime.ipynb)

<a id='creation'></a>
## Creation and uses

Dates are ordered and can be given in a list of dates, as any index, or  with the methods `pd.date_range(start, end, periode, frequence)` where 

* you should choose between end and period, period being the number of iteration
* frequence is define by these acronymes:

```
B    business day frequency
C    custom business day frequency (experimental)
D    calendar day frequency
W    weekly frequency
M    month end frequency
BM   business month end frequency
CBM  custom business month end frequency
MS   month start frequency
BMS  business month start frequency
CBMS custom business month start frequency
Q    quarter end frequency
BQ   business quarter end frequency
QS   quarter start frequency
BQS  business quarter start frequency
A    year end frequency
BA   business year end frequency
AS   year start frequency
BAS  business year start frequency
BH   business hour frequency
H    hourly frequency
T, min minutely frequency
S      secondly frequency
L, ms  milliseconds
U, us  microseconds
N      nanoseconds
```

see http://pandas.pydata.org/pandas-docs/stable/timeseries.html

<a id='creation'></a>
## Création et utilisation d'un tableau chronologique

Les dates sont ordonnées et peuvent être données dans une liste de dates, sous forme d’index, ou avec la méthode

`pd.date_range (début, fin, période, fréquence)`

où vous devez choisir entre fin et période, la période étant le nombre d'itérations

La fréquence est définie par ces codes (attention, ils sont différents de ceux de Numpy datetime) :

```
B        jours ouvrables
C        jours ouvrables personnalisée (à titre expérimental)
J        jour
W        hebdomadaire
M        mensuel
BM       mois des affaires
CBM      mois d'activité personnalisée
MS       début de mois
BMS      début de mois d'activité
CBMS     début de mois d'activité personnalisée
Q        fin de trimestre
BQ       trimestre d’affaires
QS       début de trimestre
BQS      début du trimestre d'activité
A        fin d'année
BA       fin d'année d'exercice
AS       début d'année 
BAS      année de début d'exercice
BH       heure professionel
H        heure
T, min   minute
S        seconde
L, ms    millisecondes
U, us    microsecondes
N        nanosecondes
```

In [1]:
import numpy as np
import pandas as pd
np.random.seed(1)

In [2]:
dates = pd.date_range('2016-08-28', '2016-09-06', freq='B') # begin, end, only business days
dates

DatetimeIndex(['2016-08-29', '2016-08-30', '2016-08-31', '2016-09-01',
               '2016-09-02', '2016-09-05', '2016-09-06'],
              dtype='datetime64[ns]', freq='B')

Avec cet index on peut créer un tableau chronologique :

It is easy to extract parts of a TimeSeries:

In [3]:
tdf1 = pd.DataFrame({'temperature': 20 + np.random.randint(0,5,7),
                     'pression'   : 1 + np.random.random(7)/10 },
                    index=dates)
tdf1

,temperature,pression
2016-08-29,23,1.039658
2016-08-30,24,1.038791
2016-08-31,20,1.066975
2016-09-01,21,1.093554
2016-09-02,23,1.084631
2016-09-05,20,1.031327
2016-09-06,20,1.052455


Comme pour les tableaux usuels on peut selectionner les parties qui nous intéressent avec `loc` et les filtres.
Il est également possible de contraindre les dates :

As for  usual tables we can select the parts that interest us with `loc` and filters.
It is also possible to constrain the dates:

In [4]:
tdf1.loc['2016-08']  # just August

,temperature,pression
2016-08-29,23,1.039658
2016-08-30,24,1.038791
2016-08-31,20,1.066975


In [5]:
tdf1.loc['2016-09-03':]  # after that date even if the date is not in the index

,temperature,pression
2016-09-05,20,1.031327
2016-09-06,20,1.052455


<a id='manipulation'></a>
## Manipulation

###  Boucher les trous

Soit deux sources d'information incomplètes, utilisons les méthodes que l'on a déjà vu pour boucher les trous.

###  Fill the holes

If two sources of information are incomplete, let's use methods we have already seen to fill in the gaps.

In [6]:
tdf2 = tdf1.copy()
tdf1.drop(tdf1.index[[0,1,3]], inplace=True)   # we remove some data
tdf2.drop(tdf2.index[[5,6]], inplace=True)     # more data removed
tdf2.drop(columns='pression', inplace=True)
display(tdf1, tdf2)

,temperature,pression
2016-08-31,20,1.066975
2016-09-02,23,1.084631
2016-09-05,20,1.031327
2016-09-06,20,1.052455


,temperature
2016-08-29,23
2016-08-30,24
2016-08-31,20
2016-09-01,21
2016-09-02,23


On utilse `merge` pour aggréger les données des deux tableaux. Comme on veut se baser sur la température ainsi que sur l'index, il faut passer l'index en une colonne (c'est fusion sur l'index ou sur des colonnes).

We use `merge` to aggregate the data from the two arrays. As we want to be based on the temperature as well as on the index, it is necessary to set index as a column (it is either merge on index or on columns).

In [7]:
res = pd.merge(tdf1.reset_index(), tdf2.reset_index(), on=['temperature', 'index'], how='outer')
display(res)
res.set_index('index').sort_index()

,index,temperature,pression
0,2016-08-31,20,1.066975
1,2016-09-02,23,1.084631
2,2016-09-05,20,1.031327
3,2016-09-06,20,1.052455
4,2016-08-29,23,NaN
5,2016-08-30,24,NaN
6,2016-09-01,21,NaN


,temperature,pression
index,,
2016-08-29,23,NaN
2016-08-30,24,NaN
2016-08-31,20,1.066975
2016-09-01,21,NaN
2016-09-02,23,1.084631
2016-09-05,20,1.031327
2016-09-06,20,1.052455


Si les deux sources de données ne sont pas d'accord sur une valeur, que se passe-t-il ?

If the two data sources do not agree on a value, what happens?

In [8]:
tdf1.loc['2016-08-31','temperature'] = 19
res = pd.merge(tdf1.reset_index(), tdf2.reset_index(), on=['temperature', 'index'], how='outer')
res = res.set_index('index').sort_index()
res

,temperature,pression
index,,
2016-08-29,23,NaN
2016-08-30,24,NaN
2016-08-31,19,1.066975
2016-08-31,20,NaN
2016-09-01,21,NaN
2016-09-02,23,1.084631
2016-09-05,20,1.031327
2016-09-06,20,1.052455


`merge` en mode `outer` garde toutes les valeurs, aussi a 2 températures différentes pour le 31/08.

`merge` in` outer` mode keeps all values, also at 2 different temperatures for 08/31.

### Interpolation

Comme avec Numpy, l'interpolation peut être faite en prenant en compte les dates et donc l'écart entre 2 dates successives.

### Interpolation

As with Numpy, the interpolation can be done by taking into account the dates and therefore the difference between 2 successive dates.

In [9]:
res.interpolate(method='time')

,temperature,pression
index,,
2016-08-29,23,NaN
2016-08-30,24,NaN
2016-08-31,19,1.066975
2016-08-31,20,1.066975
2016-09-01,21,1.075803
2016-09-02,23,1.084631
2016-09-05,20,1.031327
2016-09-06,20,1.052455


Il est également possible de changer l'index et de demander de recalculer les valeurs sur le nouvel index. Bizarrement cela ne marche que sur une Serie (une colonne) :

It is also possible to change the index and ask to recalculate the values ​​on the new index. Strangely, this only works on a Serie (one column):

In [10]:
tdf1['temperature'].resample('30H').interpolate('time')

2016-08-31 00:00:00    19.00
2016-09-01 06:00:00    21.50
2016-09-02 12:00:00    22.50
2016-09-03 18:00:00    21.25
2016-09-05 00:00:00    20.00
Freq: 30H, Name: temperature, dtype: float64

Pour le faire sur un DataFrame complet on peut faire les colonnes une par une ou cela :

To do it on a complete DataFrame we can interpolate columns one by one or do that:

In [11]:
interpol = tdf1.asfreq('30H')
display(interpol)
tmp = pd.concat([tdf1, interpol]).sort_index().interpolate(method='time').drop_duplicates()
tmp.loc[interpol.index]

,temperature,pression
2016-08-31 00:00:00,19.0,1.066975
2016-09-01 06:00:00,NaN,NaN
2016-09-02 12:00:00,NaN,NaN
2016-09-03 18:00:00,NaN,NaN
2016-09-05 00:00:00,20.0,1.031327


,temperature,pression
2016-08-31 00:00:00,19.00,1.066975
2016-09-01 06:00:00,21.50,1.078010
2016-09-02 12:00:00,22.50,1.075747
2016-09-03 18:00:00,21.25,1.053537
2016-09-05 00:00:00,20.00,1.031327


### Grouper les données

Depuis la version 0.18, la méthode `resample` est devenu un group-by temporel ce qui simplifie les
opérations usuelle de `groupby` sur les DataFrame indexé par le temps :

### Group data

Since version 0.18, the `resample` method has become a temporal group-by which simplifies the
usual `groupby` operations on Time-indexed DataFrame:

In [15]:
tdf1.resample('W').mean()  # to get means for each week

,temperature,pression
2016-09-04,21,1.075803
2016-09-11,20,1.041891


## Plus

Pour plus d'information sur les tableaux chronologiques on regardera la page sur les séries chronologiques :
http://pandas.pydata.org/pandas-docs/stable/timeseries.html

## More

For more information on time tables, see the page on time series:
http://pandas.pydata.org/pandas-docs/stable/timeseries.html

{{ PreviousNext("pd06 -- Merging 2 dataframes.ipynb","pd08 -- Tools.ipynb")}}